In [3]:
import os
import numpy as np

# Корневая папка
root_folder = 'aclImdb'

# Функция для загрузки данных и оценок
def load_reviews_and_ratings(folder_path):
    reviews = []
    ratings = []
    
    # Проход по файлам в папке
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            # Извлекаем оценку из имени файла (например, "номер файла_оценка фильма.txt")
            rating = int(filename.split('_')[1].split('.')[0])
            # Открываем и читаем текст файла
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                review = file.read()
            
            reviews.append(review)
            ratings.append(rating)
    
    return reviews, ratings

# Списки для хранения тренировочных и тестовых текстов и оценок
train_reviews = []
train_ratings = []
test_reviews = []
test_ratings = []

# Загрузка тренировочных данных
for label, folder in [('neg', 0), ('pos', 1)]:
    folder_path = os.path.join(root_folder, 'train', label)
    reviews, ratings = load_reviews_and_ratings(folder_path)
    train_reviews.extend(reviews)
    train_ratings.extend(ratings)

# Загрузка тестовых данных
for label, folder in [('neg', 0), ('pos', 1)]:
    folder_path = os.path.join(root_folder, 'test', label)
    reviews, ratings = load_reviews_and_ratings(folder_path)
    test_reviews.extend(reviews)
    test_ratings.extend(ratings)

# Преобразуем метки в массив NumPy для удобства
train_ratings = np.array(train_ratings)
test_ratings = np.array(test_ratings)

# Проверим, сколько данных загружено
print(f"Загружено {len(train_reviews)} тренировочных отзывов и {len(test_reviews)} тестовых отзывов")


Загружено 25000 тренировочных отзывов и 25000 тестовых отзывов


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Инициализация TF-IDF векторизатора
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)  # Ограничим количество признаков для ускорения

# Обучение на тренировочных данных и преобразование их в векторы
train_vectors = vectorizer.fit_transform(train_reviews)

# Преобразование тестовых данных в векторы (без повторного обучения)
test_vectors = vectorizer.transform(test_reviews)

# Проверим форму полученных матриц
print(f"Размерность тренировочных данных: {train_vectors.shape}")
print(f"Размерность тестовых данных: {test_vectors.shape}")


Размерность тренировочных данных: (25000, 5000)
Размерность тестовых данных: (25000, 5000)


In [32]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# Инициализация модели SVM для регрессии
svr_model = SVR(kernel='linear')

# Обучение модели на векторах
train_vectors = vectorizer.fit_transform(train_reviews)
svr_model.fit(train_vectors, train_ratings)

# Преобразование тестовых данных в векторы
test_vectors = vectorizer.transform(test_reviews)

# Предсказание оценок на тестовых данных
test_predictions = svr_model.predict(test_vectors)

# Оценка качества модели
mse = mean_squared_error(test_ratings, test_predictions)
print(f"Среднеквадратичная ошибка на тестовых данных: {mse:.4f}")

mae = mean_absolute_error(test_ratings, test_predictions)
print(f"MAE на тестовых данных: {mae:.4f}")

Среднеквадратичная ошибка на тестовых данных: 4.6769
 ошибка на тестовых данных: 1.7309


In [11]:
from sklearn.svm import SVR, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, classification_report

# Инициализация моделей
classifier_model = LogisticRegression()  # Модель для классификации

# Обучение моделей


# Для классификации: делаем метки классов (положительный = 1, отрицательный = 0)
train_labels = [1 if rating >= 5 else 0 for rating in train_ratings]
classifier_model.fit(train_vectors, train_labels)

# Преобразование тестовых данных
# Предсказание оценок на тестовых данных (регрессия)
test_predictions_ratings = svr_model.predict(test_vectors)

# Предсказание категорий на тестовых данных (классификация)
test_predictions_labels = classifier_model.predict(test_vectors)

# Оценка качества регрессии (предсказание оценок)
mse = mean_squared_error(test_ratings, test_predictions_ratings)
print(f"Среднеквадратичная ошибка на тестовых данных: {mse:.4f}")

# Оценка качества классификации (положительный или отрицательный отзыв)
test_labels = [1 if rating >= 5 else 0 for rating in test_ratings]
print(classification_report(test_labels, test_predictions_labels))

Среднеквадратичная ошибка на тестовых данных: 4.6769
              precision    recall  f1-score   support

           0       0.89      0.88      0.89     12500
           1       0.88      0.89      0.89     12500

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000



['classifier_model.pkl']